In [1]:
# Load C traces
import pandas as pd
import json
import re
from pprint import pp
from tree_sitter_languages import get_parser
import os
from lxml import etree
from pathlib import Path
import tqdm

#files = list(Path('C/Traces').glob("**/log_*.xml"))

# Define the base directory
base_dir = Path('/home/XXX/trace-modeling-oss-fuzz-c/trace_test/traces')

# List all directories ending with '_own'
own_dirs = [d for d in base_dir.glob("*_own") if d.is_dir()]

log_files = []

for trace_dir in own_dirs:
    log_files.extend(Path(trace_dir).glob("**/log_*.xml"))

print(len(own_dirs))
print(len(log_files))

84
39034


In [2]:
import json
import os
from pathlib import Path
from filelock import FileLock
import tqdm
from alias import process_xml_to_code

class GlobalFunctionTracker:
    """Persistent function tracker using source code and function input pairs"""
    def __init__(self, cache_file=".function_cache.json"):
        self.cache_file = cache_file
        self.lock = FileLock(f"{cache_file}.lock")
        self.cache = set()
        self._load_cache()
    
    def _load_cache(self):
        try:
            with self.lock:
                if os.path.exists(self.cache_file):
                    with open(self.cache_file, 'r') as f:
                        self.cache = set(json.load(f))
        except Exception as e:
            print(f"Warning: Could not load cache - {e}")
            self.cache = set()
    
    def _save_cache(self):
        try:
            with self.lock:
                with open(self.cache_file, 'w') as f:
                    json.dump(list(self.cache), f)
        except Exception as e:
            print(f"Warning: Could not save cache - {e}")
    
    def add(self, source_code, function_input):
        # Create a unique signature from the source code and input
        signature = self._generate_signature(source_code, function_input)
        if signature not in self.cache:
            self.cache.add(signature)
            self._save_cache()
            return True
        return False
    
    def contains(self, source_code, function_input):
        signature = self._generate_signature(source_code, function_input)
        return signature in self.cache
    
    def _generate_signature(self, source_code, function_input):
        """Generate a hash-based signature for the source code and input pair"""
        return f"{hash(source_code.strip())}:{hash(tuple(sorted(function_input)))}"

    def __len__(self):
        return len(self.cache)


global_tracker = GlobalFunctionTracker()
output_file = "aliasing_dataset_new.jsonl"

def is_valid_result(result):
    """Check if result contains all required fields with valid content"""
    required = {
        "Source Code": lambda x: len(x.strip()) > 0,
        "Selected Statement": lambda x: len(x.strip()) > 0,
        "Selected Pointer": lambda x: len(x.strip()) > 0,
        "Function Input": lambda x: len(x) > 0
    }
    return all(field in result and validator(result[field]) 
           for field, validator in required.items())

def process_single_file(fpath):
    src_file_path = Path(str(fpath).replace("log_", "src_"))
    
    try:
        result = process_xml_to_code(str(fpath), str(src_file_path))
        
        if result and is_valid_result(result):
            # Check if we've seen this source code + input pair before
            if global_tracker.contains(result["Source Code"], result["Function Input"]):
                return False
            
            # If not, add to dataset and tracker
            with open(output_file, 'a') as f:
                json.dump(result, f)
                f.write('\n')
            
            global_tracker.add(result["Source Code"], result["Function Input"])
            return True
        
        return False
    
    except Exception as e:
        print(f"Error processing {fpath}: {e}")
        return None

def process_files_in_batches(log_files, batch_size=8):
    """Process files in batches with progress tracking"""
    os.makedirs(os.path.dirname(output_file) or '.', exist_ok=True)
    
    # Clear output file at start
    open(output_file, 'w').close()
    
    processed_count = 0
    with tqdm.tqdm(total=len(log_files)) as pbar:
        for i in range(0, len(log_files), batch_size):
            batch = log_files[i:i+batch_size]
            
            for fpath in batch:
                success = process_single_file(fpath)
                if success:
                    processed_count += 1
                pbar.update(1)
            
            # Periodic status updates
            if i % (10*batch_size) == 0:
                print(f"\nProcessed {i}/{len(log_files)} files")
                print(f"Valid results: {processed_count}")
                print(f"Unique function/input pairs: {len(global_tracker)}")
    
    print(f"\nCompleted processing {len(log_files)} files")
    print(f"Total valid results saved to {output_file}: {processed_count}")
    print(f"Total unique function/input pairs: {len(global_tracker)}")


if __name__ == "__main__":
    process_files_in_batches(log_files, batch_size=8)

  0%|          | 0/39034 [00:00<?, ?it/s]Error processing source file: Source file not found: /home/XXX/trace-modeling-oss-fuzz-c/trace_test/traces/tarantool_own/trace_luaL_loadbuffer_fuzzer_exe/src_0.xml
Falling back to HTML parser for /home/XXX/trace-modeling-oss-fuzz-c/trace_test/traces/tarantool_own/trace_swim_proto_member_fuzzer_exe/log_0.xml
Error processing call: 'NoneType' object has no attribute 'splitlines'
Error processing call: 'NoneType' object has no attribute 'splitlines'
Error processing call: 'NoneType' object has no attribute 'splitlines'
Error processing call: 'NoneType' object has no attribute 'splitlines'
Error processing call: 'NoneType' object has no attribute 'splitlines'
Error processing call: 'NoneType' object has no attribute 'splitlines'
Error processing call: 'NoneType' object has no attribute 'splitlines'
Error processing call: 'NoneType' object has no attribute 'splitlines'
Error processing call: 'NoneType' object has no attribute 'splitlines'
Error proce

Skipping - Token count 3691 >= 2500


  0%|          | 2/39034 [00:02<10:58:40,  1.01s/it]Falling back to HTML parser for /home/XXX/trace-modeling-oss-fuzz-c/trace_test/traces/tarantool_own/trace_csv_fuzzer_exe_exe/log_2.xml
Fatal error processing trace file /home/XXX/trace-modeling-oss-fuzz-c/trace_test/traces/tarantool_own/trace_csv_fuzzer_exe_exe/log_2.xml: 'NoneType' object has no attribute 'xpath'
Falling back to HTML parser for /home/XXX/trace-modeling-oss-fuzz-c/trace_test/traces/tarantool_own/trace_swim_proto_meta_fuzzer_exe/log_0.xml
Error processing call: 'NoneType' object has no attribute 'splitlines'
Error processing call: 'NoneType' object has no attribute 'splitlines'
Error processing call: 'NoneType' object has no attribute 'splitlines'
Error processing call: 'NoneType' object has no attribute 'splitlines'
Error processing call: 'NoneType' object has no attribute 'splitlines'
Error processing call: 'NoneType' object has no attribute 'splitlines'
Error processing call: 'NoneType' object has no attribute 'spli

Skipping - Token count 3693 >= 2500


  0%|          | 8/39034 [00:05<6:34:35,  1.65it/s] 

...


Processed 14480/39034 files
Valid results: 49
Unique function/input pairs: 49


 37%|███▋      | 14571/39034 [58:19<22:26, 18.16it/s]


Processed 14560/39034 files
Valid results: 49
Unique function/input pairs: 49


 37%|███▋      | 14631/39034 [2:32:35<4:14:30,  1.60it/s] 


KeyboardInterrupt: 

In [ ]:
import sys
import json
from alias import process_xml_to_code

jsonl_file_path = "aliasing_dataset_new.jsonl"  

for fpath in log_files[:50]:
    src_file_path = str(fpath).replace("log_", "src_")
    
    result = process_xml_to_code(str(fpath), str(src_file_path))
    
    if result:
        has_code = len(result["Source Code"].strip()) > 0
        has_selected = len(result["Selected Statement"].strip()) > 0
        has_pointer = len(result["Selected Pointer"].strip()) > 0
        has_input = len(result["Function Input"]) > 0 
        
        if (has_code and has_selected and has_pointer) and has_input:
            print(result["Function Input"])
            with open(jsonl_file_path, "a") as jsonl_file:
                json_str = json.dumps(result)
                jsonl_file.write(json_str + "\n")
        else:
            print(f"Skipping incomplete result from {fpath}")

In [ ]:
import json


with open("aliasing_dataset.jsonl", "r") as f:
    lines = f.readlines()

with open("aliasing_dataset.jsonl", "w") as f:
    for idx, line in enumerate(lines):
        try:
            data = json.loads(line)
            data["idx"] = idx 
            f.write(json.dumps(data) + "\n")
        except json.JSONDecodeError:
            print(f"Skipping malformed line at position {idx}")
            continue

print(f"Added idx to {len(lines)} samples")